In [9]:
#import
from selenium import webdriver
import time
import csv
import re
import pandas as pd

In [10]:
#setup
abbreviations = [
    "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
    "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
    "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
    "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
    "WV", "WY","DC"
]
driver = webdriver.Chrome()
df = pd.DataFrame(columns=['businessName', 'address', 'city', 'state', 'postalcode', 'reportUrl', 'phone', 'location', 'rating'])
f = open('output.txt', 'w+')

In [ ]:
#main loop to go through each page filtered by state to get as much info as possible

for abr in abbreviations:
    for i in range(1,16):
        pageUrl = 'https://www.bbb.org/search?filter_state=' + abr + '&find_country=USA&find_entity=25049-000&find_id=2302_6200-3600&find_text=Payday%20Loans&find_type=Category&page=' + str(i)
        driver.get(pageUrl)
        time.sleep(3)
        text = driver.page_source
        rows = re.findall(r'{"sharedGoogleAnalyticsId":(?:.|\n)*?totalResults', text)

        #check if it is a page with results
        if len(rows) < 2:
            break
        rows = rows[1]
        if ('"title":"No Results"' in rows):
            break

        rows = rows.split("accreditationStatus")
        for row in rows[:-1]:

            businessName = 'null'
            if('"businessName":"' in row):
                businessName = re.findall(r'"businessName":"(?:.|\n)*?"', row)[0][16:-1]
                #print(businessName)

            address = 'null'
            if('"address":"' in row):
                address = re.findall(r'"address":"(?:.|\n)*?"', row)[0][11:-1]
                #print(address)

            city = 'null'
            if('"city":"' in row):
                city = re.findall(r'"city":"(?:.|\n)*?"', row)[0][8:-1]
                #print(city)
            
            state = 'null'
            if('"state":"' in row):
                state = re.findall(r'"state":"(?:.|\n)*?"', row)[0][9:-1]
                #print(state)

            postalcode = 'null'
            if('"postalcode":"' in row):
                postalcode = re.findall(r'"postalcode":"(?:.|\n)*?"', row)[0][14:-1]
                #print(postalcode)

            reportUrl = 'null'
            if('"reportUrl":"' in row):
                reportUrl = re.findall(r'"reportUrl":"(?:.|\n)*?"', row)[0][13:-1]
                #print(reportUrl)

            phone = 'null'
            if('"phone":[' in row):
                phone = re.findall(r'"phone":\[(?:.|\n)*?\]', row)[0][8:]
                #print(phone)

            location = 'null'
            if('"location":"' in row):
                location = re.findall(r'"location":"(?:.|\n)*?"', row)[0][13:-1]
                #print(location)

            rating = 'null'
            if('"rating":"' in row):
                rating = re.findall(r'"rating":"(?:.|\n)*?"', row)[0][10:-1]
                #print(rating)

            #insert into dataframe
            df.loc[len(df)] = [businessName, address, city, state, postalcode, reportUrl, phone, location, rating]
        print(abr + str(i) + ',' + str(len(rows)-1))
        f.write(abr + str(i) + ',' + str(len(rows)-1) + '\n')

AR1,5
AZ1,15
AZ2,15
AZ3,15
AZ4,15
AZ5,15
AZ6,15
AZ7,15
AZ8,15
AZ9,15
AZ10,15
AZ11,15
AZ12,15
AZ13,15
AZ14,15
AZ15,15
CA1,15
CA2,15
CA3,15
CA4,15
CA5,15
CA6,15
CA7,15
CA8,15
CA9,15
CA10,15
CA11,15
CA12,15
CA13,15
CA14,15
CA15,15
CO1,15
CO2,15
CO3,15
CO4,15
CO5,15
CO6,15
CO7,2
DE1,15
DE2,15
DE3,15
DE4,15
DE5,15
DE6,10
FL1,15
FL2,15
FL3,15
FL4,15
FL5,15
FL6,15
FL7,15
FL8,15
FL9,15
FL10,15
FL11,15
FL12,15
FL13,15
FL14,15
FL15,15
GA1,15
GA2,15
GA3,15
GA4,15
GA5,15
GA6,15
GA7,15
GA8,15
GA9,15
GA10,15
GA11,15
GA12,15
GA13,15
GA14,15
GA15,15


In [ ]:
#export scrape to csv
df.to_csv('bbbScrape.csv')